In [1]:
TODO='''next TODO: 
we have errors of types mismatch (like at wrealsum to wrealavg), bus of logic to real bus, and warnings real to wire
they have all the same pattern - file path, net name, bus size etc.
so need to unite all the issues into 1 single table
then turn this table into sed command at the end
'''
TODO ='''
sed -ri '8s@((input|output|inout)\s+)?(wire\s+)?(logic\s+)?\s*ldo_th2_boost_0p7_out@\1wrealsum ldo_th2_boost_0p7_out@' '/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5brk2adcldo_ldo_top.v' 
sed -ri '8s@((input|output|inout)\s+)?(wire\s+)?(logic\s+)?\s*ldo_th2_boost_0p7_out@\1wrealavg ldo_th2_boost_0p7_out@' '/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5brk2adcldo_ldo_top.v' 

input:
    output   ldo_th2_boost_0p7_out, 
output
    output   wrealsumwrealavg ldo_th2_boost_0p7_out,

'''

TODO='''
at pctm sometimes i have 
        Expression:  vofc_q1q3s1s2sl2sl5_n[3]
        Port:        wire vout_n;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/1.small_fixes/ipn5adcrefgenshr_mux.v",
      10
      Nettype nets cannot be connected to builtin net types.
and sometimes it's Expression with declared at after it
'''

In [2]:
if 'log' not in list(locals().keys())+list(globals().keys()):
    log='/nfs/iil/disks/falcon_tc2-rtl/users/lisrael1/first/analog_sa/src/bmod/brk_adc_top/sim/log'
    # log='/nfs/iil/disks/falcon_tc2-rtl/users/lisrael1/first/analog_sa/src/bmod/brk_adc_fe/netlist_steps/sim/log'
    log='/nfs/iil/disks/falcon_tc2-rtl/users/lisrael1/first/analog_sa/src/bmod/brk_adc_top/sim_with_fe/log'
    log='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/log3'
    log='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/sim/log'
    log='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/5.brk_gen2_1p0/sim/log'
    log='/nfs/iil/disks/barak_gen2_rtl/users/lisrael1/first/run_sim_prod/log'
    log='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p0/sim/log'
    log='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1/sim/log'

In [3]:
from IPython.display import display
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",1000) # don’t put … instead of multi columns
pd.set_option('expand_frame_repr',False) # for not wrapping columns if you have many
pd.set_option("display.max_rows",1000)
pd.set_option('display.max_colwidth',10000)

from IPython.display import HTML
def header(text, size=100):
#     print('*'*size)
#     print("{:{fill}{align}{width}}".format("  "+text+"  ", fill='*', align='^', width=size))
#     print('*'*size)
    html='''
    <p style="font-size:32px; color:#cc5c00; font-family:consolas;"> 
            <br>
            {text}
            <br>
            <br>
    </p>'''.format(text=text)
    display(HTML(html))

def print_linux_windows_path(linux_path, prefix='\t'):
    print(prefix, linux_path)
    win=r'\\isamba.iil.intel.com'+linux_path.replace('/','\\')
#     win=!winpath.sh $linux_path
    print(prefix, win)
    
output_directory='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_tmp/'
output_directory='/nfs/iil/disks/hip_ana_sim_01/dgottesm/analysis_and_tools/voila/real_to_wire_issues/results/'


/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/virtual_env/env/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
header('errors')

In [5]:
errors = !cat $log | grep -P 'Error|Warning|Lint' | sort | uniq -c |sort -nr
print('\n'.join(errors))
cmd=f'''grep URMI -A 2 {log} | grep -Pv 'Error|--' | tr -d ',"' | awk '{{print $1}}' | paste - - | grep -oP '/[^/]+\s\S+$' | uniq -c | sort -nr | column -t'''
missing_blocks = !$cmd
if len(missing_blocks):
    print('\n\nmissing blocks:')
    print('\n'.join(missing_blocks))
    print(f'\n\n\tcmd: \n\t{cmd}\n')


    104 Lint-[PCWM-L] Port connection width mismatch
     56 Warning-[IPC] Illegal high conn for port connect
     48 Error-[PCTM] Port connection type mismatch
      3 Warning-[NMIN] No module instance name
      2 Lint-[TFIPC-L] Too few instance port connections
      2 Error-[EIRO] Illegal real operand
      1 Warning-[LCA_FEATURES_ENABLED] Usage warning
      1 Warning-[DEBUG_DEP] Option will be deprecated
      1 Error-[VIPCBD] Variable input ports cannot be driven


In [6]:
' common errors '
error_types=pd.DataFrame(index=['meaning'])
error_types.index.name='error code'
error_types['ICSD,ICPSD']='multi driver on real'
error_types['PCTM']='packed connected to unpacked or wreal connected to wire'
error_types['VIHIOP']="real cannot be connected to inout port, even if it's wreal type"
error_types['IAP']="you used '{} instead of {} or you assing to only part of the bus. dimension issue"
error_types['UNTPC']="you used {} instead of ''{} at bus of wreal"
error_types.T

error code,meaning
"ICSD,ICPSD",multi driver on real
PCTM,packed connected to unpacked or wreal connected to wire
VIHIOP,"real cannot be connected to inout port, even if it's wreal type"
IAP,you used '{} instead of {} or you assing to only part of the bus. dimension issue
UNTPC,you used {} instead of ''{} at bus of wreal


In [7]:
content=open(log).read()
df=pd.Series(content.split('\n\n\n')).to_frame('original')
df['error']=df.original.str.startswith('Error')
df['Lint']=df.original.str.startswith('Lint')
df['stam']=1
df.update(df.query('error').assign(stam=2)[['stam']])

In [8]:
header('list of compiled files')

In [9]:
' note that if you have multiple files at the same name, it will take the first one, even if you have ifdef that takes the second one... '
files=pd.Series(content.split('Top Level Modules')[0].split('\n')).str.extract("Parsing design file '(?P<file_path>[^']+)'").dropna().reset_index(drop=True)
files['module_name']=files.file_path.str.rsplit('/',1).str[-1].str.rsplit('.',1).str[0]
files=files.drop_duplicates('module_name', keep='first')
display(files.set_index('module_name').head(10).style.set_properties(**{'text-align': 'left'}))
print('you have %d files'%files.shape[0])
print_linux_windows_path(output_directory+"compiled_files.csv")
files.to_csv(output_directory+'compiled_files.csv')

,file_path
module_name,
snps_msv_nettype_pkg,/p/hdk/cad/vcsmx/P-2019.06-SP2-3/etc/snps_msv/snps_msv_nettype_pkg.svp
importing_wrealsum,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/importing_wrealsum.v
msv_buffer_delay,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_buffer_delay.sv
msv_clock_freq_multipy,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_clock_freq_multipy.sv
msv_gain_with_common,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_gain_with_common.sv
msv_phase_delay_two_clocks,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_phase_delay_two_clocks.sv
msv_phases_splitter,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_phases_splitter.sv
msv_rc_filter,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_rc_filter.sv
msv_serializer,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//extra_bmods/msv_serializer.sv


you have 601 files
	 /nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_tmp/compiled_files.csv
	 \\isamba.iil.intel.com\nfs\iil\disks\ams_regruns\dkl\users\lisrael1\msv\temp_del\verilog_tmp\compiled_files.csv


In [10]:
header('multi driver that need to be changed to wreal')

In [11]:
error_multidriver_pattern = r'Variable "(?P<multidriven_name>\w+)" is driven by multiple structural drivers.*This variable is declared at "(?P<file_path>[^"]+)"'.replace(' ','\s+')

multidriven=df.query('error').original.str.replace('\n',' ').str.extract(error_multidriver_pattern).drop_duplicates().reset_index(drop=True)
multidriven['content']="var real "+multidriven.multidriven_name
if not multidriven.dropna().empty:
    display(multidriven.dropna().head(1).T)
    multidriven.to_csv(output_directory+'multi_drivers.csv')
    print_linux_windows_path(output_directory+"multi_drivers.csv")
else:
    print('no multi driver issue')

no multi driver issue


In [12]:
# dcmon=wreal.dropna().query('wreal_name.str.contains("dcmon")')  # ' automatically change to wreal at dcmon '
command="sed -ri 's="+multidriven.dropna().content.replace(r'var ',r'(var\s*|)',regex=True).replace(' ','\s+',regex=True)+"="+multidriven.dropna().content.replace('var real ',' wreal ', regex=True)+"=' "+multidriven.dropna().file_path
command=command.unique()
number_of_replacments=command.shape
command="\n".join(command.tolist())
command_output=output_directory+'multidriven_to_wreal.sh'
f=open(command_output, 'w')
f.write(command)
f.close()
!chmod o+r $command_output
if not multidriven.dropna().empty:
    print('now run\n\tbash '+command_output)
    print('%d replacments'%number_of_replacments)

In [13]:
header('error on real bus connected to wire bus<br>or wreal connected to others')

In [14]:
errors_types_ignore='''

Expression is the local net and Port is for remote net

for changing local net you will get:
    Error-[PCTM] Port connection type mismatch
    /nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5adcsararrayshr_channel_dfx_sw.v, 61
    "ipn5anashr_dc_svt_6n_pull_vref vref_sw_sararray_p( .sig (i_sararray_dcmon_p),  .comp_en (i_sararray_comp_p_en),  .dcmon_en (i_sararray_dcmon_p_en),  .vcc_i (vcc0p9_lv),  .vref_inp (verf),  .vssx (vssx));"
      The following expression is illegally connected to port "sig" of module 
      "ipn5anashr_dc_svt_6n_pull_vref", instance "vref_sw_sararray_p". The type of
      the port does not match that of the port connect.
        Expression:  wire i_sararray_dcmon_p;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5adcsararrayshr_channel_dfx_sw.v",
      29
        Port:        wrealsum sig;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5anashr_dc_svt_6n_pull_vref.sv",
      9
      Port connections with one nettype end require matching types.
    
for changing remote net you will get:
    Error-[PCTM] Port connection type mismatch
    /nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5adcrefgenshr_mux_res_dfx.v, 100
    "ipn5adcrefgenshr_mux slc2_q3_qrt3( .dcmon_vout_n (dcmon_out1),  .dcmon_vout_p (dcmon_out2),  .vout_n (vofc_q1q3s1s2sl2sl5_n[3]),  .vout_p (vofc_q1q3s1s2sl2sl5_p[3]),  .addr (reg11_addr[31:24]),  .dcmon_sel_vout_n (dcmon_out_sel[0]),  .dcmon_sel_vout_p (dcmon_out_sel[0]),  .res_high (vres_high),  .res_low (vres_low),  .res_vcm (vcm_out),  .vcca (vcc0p9),  .vssx (vssx));"
      The following expression is illegally connected to port "vout_n" of module 
      "ipn5adcrefgenshr_mux", instance "slc2_q3_qrt3". The type of the port does 
      not match that of the port connect.
        Expression:  vofc_q1q3s1s2sl2sl5_n[3]
        Port:        wire vout_n;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5adcrefgenshr_mux.v",
      10
      Nettype nets cannot be connected to builtin net types.
    
another remote pattern that is like local pattern:
    Error-[PCTM] Port connection type mismatch
    /nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5brk2adctop_adc_top.v, 952
    "ipn5anashr_dec_adc_channel_top channel_decoder( .add_domain_dis0 (channel_decoder_add_domain_dis0),  .add_domain_dis1 (channel_decoder_add_domain_dis1),  .div_readsel (div_readsel),  .div_strb (div_strb),  .ffe_strb_all_in (channel_decoder_ffe_strb_all_in),  .o_adcfe_read_data (channel_decoder_o_adcfe_read_data),  .o_add_ffe (channel_decoder_o_add_ffe),  .o_add_out_to_rx (o_adc_add),  .o_add_pi_prebuff (channel_decoder_o_add_pi_prebuff),  .o_channel_comp_n (o_adc_compmon_n),  .o_channel_comp_p (o_adc_compmon_p),  .o_channel_dcmon_n (o_adc_adcfe_dcmon_n),  .o_channel_dcmon_p (o_adc_adcfe_dcmon_p),  .o_dig_spare_to_sar (channel_decoder_o_dig_spare_to_sar),  .o_div_rstb (div_rstb),  .o_div_write_data (div_write_data),  .o_hsmon_local (o_adc_hsmon_local),   ... "
      The following expression is illegally connected to port 
      "i_sar_array_comp_n_dfx" of module "ipn5anashr_dec_adc_channel_top", 
      instance "channel_decoder". The type of the port does not match that of the 
      port connect.
        Expression:  wrealsum sar_array_o_sar_array_comp_n_dfx;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5brk2adctop_adc_top.v",
      215
        Port:        wire i_sar_array_comp_n_dfx;
        Declared at: 
      "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/2.propagating_reals/ipn5anashr_dec_adc_channel_top.v",
      58
      Nettype nets cannot be connected to builtin net types.
      
issues that you have with the remote:
    you dont have the declared at the local file. you dont need this but you cannot have the same pattern as local pattern
    the bus size may be only part of it
    you dont have the type of the local one 

'''

In [15]:
error_bus_pattern = r'Error-\[PCTM\] Port connection type mismatch (?P<file_path>[^,]+).*Expression: (?P<local_bus_expression>[^;]+); Declared at: "(?P<local_file_path>[^"]+)", (?P<local_file_declared_line>\d+).* Port: (?P<remote_bus_expression>[^;]+); Declared at: "(?P<remote_file_path>[^"]+)", (?P<remote_file_declared_line>\d+)'.replace(' ','\s*')

local_bad_type=df.query('error').original.str.replace('\n',' ').str.extract(error_bus_pattern).drop_duplicates().dropna().reset_index(drop=True)
local_bad_type['local_bus_type']=local_bad_type.local_bus_expression.str.split(' ').str[0]
local_bad_type['remote_bus_type']=local_bad_type.remote_bus_expression.str.split(' ').str[0]
local_bad_type['local_net_name']=local_bad_type.local_bus_expression.str.replace('\[\d+:\d+\]','').str.split(' ').str[-1]
local_bad_type['remote_net_name']=local_bad_type.remote_bus_expression.str.replace('\[\d+:\d+\]','').str.split(' ').str[-1]
local_bad_type['local_bus_shape']=local_bad_type.local_bus_expression.str.extract('(?P<bus_shape>\[[\d:]+\])').bus_shape.fillna("")
local_bad_type['remote_bus_shape']=local_bad_type.remote_bus_expression.str.extract('(?P<bus_shape>\[[\d:]+\])').bus_shape.fillna("")

remote1_bad_type=local_bad_type.query('remote_bus_type=="wire"')
local_bad_type=local_bad_type.query('remote_bus_type!="wire"')

local_bad_type['change_remote_or_local']='local'
local_bad_type['line_to_update']=local_bad_type.local_file_declared_line

remote1_bad_type['change_remote_or_local']='remote'
remote1_bad_type['line_to_update']=remote1_bad_type.remote_file_declared_line
remote1_bad_type


,file_path,local_bus_expression,local_file_path,local_file_declared_line,remote_bus_expression,remote_file_path,remote_file_declared_line,local_bus_type,remote_bus_type,local_net_name,remote_net_name,local_bus_shape,remote_bus_shape,change_remote_or_local,line_to_update
9,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2rx_top.v,wrealsum pr_dc_odac_in100u,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2rx_top.v,366,wire pr_dc_odac_in100ua,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adctop_adc_top.v,148,wrealsum,wire,pr_dc_odac_in100u,pr_dc_odac_in100ua,,,remote,148
16,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,real pr0_iodac[1:0],/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,11,wire [1:0] pr0_iodac,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc.v,10,real,wire,pr0_iodac,pr0_iodac,[1:0],[1:0],remote,10
17,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,real pr1_iodac[1:0],/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,14,wire [1:0] pr1_iodac,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc.v,13,real,wire,pr1_iodac,pr1_iodac,[1:0],[1:0],remote,13
18,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,real pr2_iodac[1:0],/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,17,wire [1:0] pr2_iodac,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc.v,16,real,wire,pr2_iodac,pr2_iodac,[1:0],[1:0],remote,16
19,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,real pr3_iodac[1:0],/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc_top.v,20,wire [1:0] pr3_iodac,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2adcnsf_instage_dc.v,19,real,wire,pr3_iodac,pr3_iodac,[1:0],[1:0],remote,19
20,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2v1adcclkgen_divtop.v,wrealsum o_adc_top_dcmon_p,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2v1adcclkgen_divtop.v,15,wire o_adc_top_dcmon_p,/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/flc/1p1//3.fixing_reals/ipn5flctc2v1adcclkgen_clk_path.v,22,wrealsum,wire,o_adc_top_dcmon_p,o_adc_top_dcmon_p,,,remote,22


In [16]:
error_bus_pattern = r'Error-\[PCTM\] Port connection type mismatch (?P<local_file_path>[^,]+).*Expression: (?P<local_bus_expression>[^PD\t]+) .* Port: (?P<remote_bus_expression>[^;]+); Declared at: "(?P<remote_file_path>[^"]+)", (?P<remote_file_declared_line>\d+)'.replace(' ','\s*')

remote_bad_type=df.query('error').original.str.replace('\n',' ').str.extract(error_bus_pattern).drop_duplicates().dropna().reset_index(drop=True)
remote_bad_type=remote_bad_type.query('not local_bus_expression.str.contains(";")', engine='python')  # at the patterm we also catching the local_bad_type, so this way we remove them

remote_bad_type['local_bus_type']=remote_bad_type.local_bus_expression.str.split(' ').str[0]
remote_bad_type.local_bus_type=np.where(remote_bad_type.local_bus_type.str.contains('\['),'real',remote_bad_type.local_bus_type)
remote_bad_type['remote_bus_type']=remote_bad_type.remote_bus_expression.str.split(' ').str[0]
remote_bad_type['local_net_name']=remote_bad_type.local_bus_expression.str.replace('\[\d+:\d+\]','').str.split(' ').str[-1]
remote_bad_type['remote_net_name']=remote_bad_type.remote_bus_expression.str.replace('\[\d+:\d+\]','').str.split(' ').str[-1]

remote_bad_type['local_bus_shape']=remote_bad_type.local_bus_expression.str.extract('(?P<bus_shape>\[[\d:]+\])').bus_shape.fillna("")
remote_bad_type.local_bus_shape=np.where(remote_bad_type.local_bus_shape.str.contains(':'),remote_bad_type.local_bus_shape,"")

remote_bad_type['remote_bus_shape']=remote_bad_type.remote_bus_expression.str.extract('(?P<bus_shape>\[[\d:]+\])').bus_shape.fillna("")
remote_bad_type['change_remote_or_local']='remote'
remote_bad_type['line_to_update']=remote_bad_type.remote_file_declared_line

# remote_bad_type.loc[remote_bad_type.local_bus_type=='wire','command']=remote_bad_type.apply(lambda r:r'''sed -ri 's@(wire)?\s*{local_bus_shape}\s*{local_net_name}@{remote_bus_type} {local_net_name}{local_bus_shape}@' '{local_file_path}' '''.format(**r), axis=1)
# remote_bad_type.iloc[[1,-1]].drop(['local_file_path','remote_file_path'], axis=1).T

In [17]:
wreal=pd.concat([remote_bad_type, remote1_bad_type, local_bad_type], axis=0, sort=False)

' removing 2 updates at the same net and file (probably you have fater and son that driving 2 values)'
# wreal['change_remote_or_local']=np.where(wreal.local_bus_type=='wire','local',np.where(wreal.remote_bus_type=='wire','remote','none'))
wreal['file_to_update']=np.where(wreal.local_bus_type=='wire',wreal.local_file_path,np.where(wreal.remote_bus_type=='wire',wreal.remote_file_path,'none'))
wreal['net_to_update']=np.where(wreal.local_bus_type=='wire',wreal.local_net_name,np.where(wreal.remote_bus_type=='wire',wreal.remote_net_name,'none'))

print('found total of %d PCTM errors'%wreal.shape[0])

if wreal.shape[0]:
    concatenated_bus_to_replace=wreal.query('local_bus_type.str.contains("{")', engine='python')
    if concatenated_bus_to_replace.shape[0]:
        print('removing 1 line that has concatenation as input')
        wreal=wreal.drop(concatenated_bus_to_replace.index, axis=0)
    non_wires_net_to_update = wreal.query('net_to_update=="none"')
    if non_wires_net_to_update.shape[0]:
        print('you have {number} errors that dosnt have wire to update (like wrealsum connected to wrealavg)'.format(number=non_wires_net_to_update.shape[0]))
        non_wires_net_to_update.to_csv(output_directory+'non_wires_updates.csv')
        print('removing from update command, you can see them at:')
        print_linux_windows_path(output_directory+'non_wires_updates.csv')
        wreal=wreal.drop(non_wires_net_to_update.index, axis=0)
    duplicated_update_on_same_net = wreal.loc[wreal.duplicated(['file_to_update','net_to_update'], keep=False)]
    if duplicated_update_on_same_net.shape[0]:
        print('you have %d multi updates on the same nets. taking only %d that are the first unique ones'%(duplicated_update_on_same_net.shape[0], (~wreal.duplicated(['file_to_update','net_to_update'], keep='first')).sum()))
        duplicated_update_on_same_net.to_csv(output_directory+'duplicated_updates_on_the_same_net.csv')
        print_linux_windows_path(output_directory+'duplicated_updates_on_the_same_net.csv')
        wreal=wreal.drop_duplicates(['file_to_update','net_to_update'], keep='first')
    print('we left with %d PCTM errors'%wreal.shape[0])
        # this one has problem if the real side is vin[8]
    # wreal['command']=wreal.apply(lambda x:r'''sed -ri '{<local>_file_declared_line}s@((input|output|inout)\s+)?(wire\s+)?(logic\s+)?{<local>_bus_shape}\s*{<local>_net_name}@\1{<remote>_bus_type} {<local>_net_name}{<local>_bus_shape}@' '{<local>_file_path}' '''.replace('<local>',['remote','local'][x.local_bus_type=='wire']).replace('<remote>',['local','remote'][x.local_bus_type=='wire']).format(**x), axis=1)
#     wreal.command.replace(['wrealsum','wreal1driver','wrealavg','wreal','wrealmin','wrealmax'],'real')  # we dont want to propagate the wreal. we want to cast it to real
#     wreal.command=wreal.command.str.replace(r'\s*\s*',r'\s*', regex=False).str.replace(r'[',r'\[',1).str.replace(r']',r'\]',1)
#     wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@((input|output|inout)\s+)?((wire|logic|wreal|wrealsum|wreal1driver|wrealavg|var|real)\s+)*?(\[[\d+:]+\]\s+)?{net_to_update}@\1 real {net_to_update} \3@' '{file_to_update}' '''.format(**x), axis=1)
#     wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@(input\b|output\b|inout\b)?.*(\[[\d+:]+\]\s+)?{net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
#     wreal['command']=wreal.apply(lambda x:r'''perl '{line_to_update}s@((input|output|inout)\s+)?.*(\[[\d+:]+\]\s+)?{net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
#     wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@((input|output|inout)\s+)?.*?(\[[0-9:]+\]\s+)?{net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
if wreal.shape[0]:
#   wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@(input\s+|output\s+|inout\s+|)[^[]*(\[[0-9:]+\]\s+|\s+){net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
    # we take the specific row, so we dont can too much on the content
    # TODO - i just changed it from (\[[0-9:]+\]\s+|\s+|) to (\[[0-9:]+\]\s+|) - need to see if it works!!!!!!!!!!!!!!!
    # wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@(input\s+|output\s+|inout\s+|)[^[]*(\[[0-9:]+\]\s+|\s+|)\b{net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
    wreal['command']=wreal.apply(lambda x:r'''sed -ri '{line_to_update}s@(input\s+|output\s+|inout\s+|)[^[]*(\[[0-9:]+\]\s+|)\b{net_to_update}@\1 real {net_to_update} \2@' '{file_to_update}' '''.format(**x), axis=1)
    wreal=wreal.reset_index(drop=True)
    try:
        wreal.loc[(wreal.local_bus_type!='wire')&(wreal.remote_bus_type!='wire'),'command']='# no wire type ' +wreal.command
        wreal.loc[wreal.file_to_update.str.endswith(".sv"),'command']='# sv file ' +wreal.command
    except: pass


found total of 24 PCTM errors
you have 2 errors that dosnt have wire to update (like wrealsum connected to wrealavg)
removing from update command, you can see them at:
	 /nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_tmp/non_wires_updates.csv
	 \\isamba.iil.intel.com\nfs\iil\disks\ams_regruns\dkl\users\lisrael1\msv\temp_del\verilog_tmp\non_wires_updates.csv
you have 3 multi updates on the same nets. taking only 20 that are the first unique ones
	 /nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_tmp/duplicated_updates_on_the_same_net.csv
	 \\isamba.iil.intel.com\nfs\iil\disks\ams_regruns\dkl\users\lisrael1\msv\temp_del\verilog_tmp\duplicated_updates_on_the_same_net.csv
we left with 20 PCTM errors


In [18]:
if wreal.shape[0]:
    display(wreal.head(2).T.style.set_properties(**{'text-align': 'left'}))
    wreal.to_csv(output_directory+'type_error.csv')
    print_linux_windows_path(output_directory+"type_error.csv")
else:
    print("didn't find PCTM errors")

ValueError: style is not supported for non-unique indices.

In [ ]:
if wreal.shape[0]:
    command=wreal.command.unique()
    print('%d replacments'%command.shape)
    command="\n".join(command.tolist())
    command_output=output_directory+'type_error.sh'
    f=open(command_output, 'w')
    f.write(command)
    f.close()
    !chmod o+r $command_output
    print('now run\n\tbash '+command_output)

In [ ]:
header('warning on real connected to wire')

In [ ]:
lint_pattern=r'Port connection width mismatch (?P<local_file_path>[^,]+), (?P<file_path_line>\d+).*The following (?P<local_bit>\d+)-bit expression is connected to (?P<remote_bit>\d+)-bit port "(?P<remote_port>\w+)" of module "(?P<remote_module>\w+)", instance "(?P<remote_instance>\w+)". Expression: (?P<local_wire_name>[\w\[\]]+) The expression is from module (?P<local_module_name>\w+)'.replace(' ','\s+')
lint=df.query('Lint').original.str.replace('\n', ' ', regex=True).str.extract(lint_pattern).dropna()
lint.local_wire_name=lint.local_wire_name.replace('\[.*]', '', regex=True)
# local_lint = lint#.query('local_bit=="1"')#.drop_duplicates(subset=['local_wire_name','local_module_name','local_bit'])
lint['remote_file_path']=lint.merge(files, left_on='remote_module', right_on='module_name', how='left').file_path.values
lint['file_path_that_need_update']=lint.local_file_path.where(lint.local_bit=='1',lint.remote_file_path)
lint['net_that_need_update']=lint.local_wire_name.where(lint.local_bit=='1',lint.remote_port)

In [ ]:
display(lint.drop(['local_file_path','remote_file_path'],axis=1).head(1).T.style.set_properties(**{'text-align': 'left'}))
real_to_wire_warning_csv=output_directory+'real_to_wire_warning.csv'
lint.to_csv(real_to_wire_warning_csv)
!chmod 777 $real_to_wire_warning_csv
print_linux_windows_path(real_to_wire_warning_csv)

In [ ]:
' ** the command maker at bus real is bettern that here ** '
if not lint.empty:
    command = lint.file_path_that_need_update.str.endswith(".sv").apply(lambda r: '# sv file ' if r else '')+"perl -pi.bak -e 's=wire\s+(logic\s+)?"+lint.net_that_need_update+"=real "+lint.net_that_need_update+"=;s=(input|output)\s+(wire\s+)?(logic\s+)?"+lint.net_that_need_update+"=\\1 real "+lint.net_that_need_update+"=' "+lint.file_path_that_need_update
    command=command.dropna().unique()
    print('%d replacments'%command.shape)
    command="# sed '/vcc\|vss/d' \n"+"\n".join(command.tolist())
    command_output=output_directory+'type_warning.sh'
    f=open(command_output, 'w')
    f.write(command)
    f.close()
    !chmod o+r $command_output
    print('now run\n\tbash '+command_output)
    print_linux_windows_path(output_directory+"type_warning.csv")
    lint.to_csv(output_directory+'type_warning.csv')
else:
    print("didn't find lint issues at the log")

In [ ]:
# for i,c in enumerate(command.tolist()):
#     if type(c) is float:
#         print(i)
# # lint.iloc[49].to_frame()
# l=lint.iloc[[49]]
# l.file_path_that_need_update.str.endswith(".sv").apply(lambda r: '# ' if r else '')+"perl -pi.bak -e 's=wire\s+(logic\s+)?"+l.net_that_need_update+"=real "+l.net_that_need_update+"=;s=(input|output)\s+(wire\s+)?(logic\s+)?"+l.net_that_need_update+"=\\1 real "+l.net_that_need_update+"=' "+l.file_path_that_need_update